<a href="https://colab.research.google.com/github/Lauro-Solia/PO1.5/blob/main/PO2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Consegues ver isso, Ivanzinho??

In [2]:
!pip install simpy
import simpy
import numpy as np
import csv

In [3]:
np.random.seed(622514664)

In [4]:
class Solution(object):
  def __init__(self, env, estoqueInicialLeiteDesnatadoEmbalado,
               estoqueInicialLeiteSemidesnatadoEmbalado,
               estoqueInicialAchocolatadoDesnatadoEmbalado,
               estoqueInicialAchocolatadoSemidesnatadoEmbalado,
               estoqueInicialEmbalagemDeDesnatado,
               estoqueInicialEmbalagemDeSemidesnatado,
               estoqueInicialEmbalagensDeAchocolatadoDesnatado,
               estoqueInicialEmbalagensDeAchocolatadoSemidesnatado,
               estoqueInicialLeiteCru,
               estoqueInicialAçucarEChocolate,
               getTempoPasteurização,
               LIMITE_MAQUINA_PASTEURIZADORA,
              #  getTempoLimparMaquinaPasteurização,
               getTempoCentrifugaçãoSemidesnatado,
               getTempoCentrifugaçãoDesnatado,
               getTempoAdiçãoAçucarEChocolate,
               getTempoHomogeneização,
               getTempoEnvaseLeite,
               getTempoEnvaseAchocolatado,
               getTempoDeEntregaLeiteCruComprado,
               quantidadeMaquinasPasteurizadoras,
               quantidadeMaquinasCentrífugas,
               quantidadeContainers,
               quantidadeMaquinasEnvasadoras,
               getTempoDeEsperaEntreClientes,
               demandaSemanalLitrosDeLeite,
               pontoDeRecompraDeEstoque
               ):
    self.env = env
    self.action = env.process(self.run())

    self.estoqueLeiteDesnatadoEmbalado = estoqueInicialLeiteDesnatadoEmbalado
    self.ESTOQUE_MAXIMO_LEITE_DESNATADO_EMBALADO = estoqueInicialLeiteDesnatadoEmbalado
    self.ESTOQUE_MINIMO_LEITE_DESNATADO_EMBALADO = estoqueInicialLeiteDesnatadoEmbalado * pontoDeRecompraDeEstoque
    self.estoqueLeiteSemidesnatadoEmbalado = estoqueInicialLeiteSemidesnatadoEmbalado
    self.ESTOQUE_MAXIMO_LEITE_SEMIDESNATADO_EMBALADO = estoqueInicialLeiteSemidesnatadoEmbalado
    self.ESTOQUE_MINIMO_LEITE_SEMIDESNATADO_EMBALADO = estoqueInicialLeiteSemidesnatadoEmbalado * pontoDeRecompraDeEstoque
    self.estoqueAchocolatadoSemidesnatadoEmbalado = estoqueInicialAchocolatadoSemidesnatadoEmbalado
    self.ESTOQUE_MAXIMO_ACHOCOLATADO_SEMIDESNATADO_EMBALADO = estoqueInicialAchocolatadoSemidesnatadoEmbalado
    self.ESTOQUE_MINIMO_ACHOCOLATADO_SEMIDESNATADO_EMBALADO = estoqueInicialAchocolatadoSemidesnatadoEmbalado * pontoDeRecompraDeEstoque
    self.estoqueAchocolatadoDesnatadoEmbalado = estoqueInicialAchocolatadoDesnatadoEmbalado
    self.ESTOQUE_MAXIMO_ACHOCOLATADO_DESNATADO_EMBALADO = estoqueInicialAchocolatadoDesnatadoEmbalado
    self.ESTOQUE_MINIMO_ACHOCOLATADO_DESNATADO_EMBALADO = estoqueInicialAchocolatadoDesnatadoEmbalado * pontoDeRecompraDeEstoque
    self.estoqueEmbalagemDeDesnatado = estoqueInicialEmbalagemDeDesnatado
    self.ESTOQUE_MAXIMO_EMBALAGEM_DE_DESNATADO = estoqueInicialEmbalagemDeDesnatado
    self.ESTOQUE_MINIMO_EMBALAGEM_DE_DESNATADO = estoqueInicialEmbalagemDeDesnatado * pontoDeRecompraDeEstoque
    self.estoqueEmbalagemDeSemidesnatado = estoqueInicialEmbalagemDeSemidesnatado
    self.ESTOQUE_MAXIMO_EMBALAGEM_DE_SEMIDESNATADO = estoqueInicialEmbalagemDeSemidesnatado
    self.ESTOQUE_MINIMO_EMBALAGEM_DE_SEMIDESNATADO = estoqueInicialEmbalagemDeSemidesnatado * pontoDeRecompraDeEstoque
    self.estoqueEmbalagensDeAchocolatadoSemidesnatado = estoqueInicialEmbalagensDeAchocolatadoSemidesnatado
    self.ESTOQUE_MAXIMO_EMBALAGENS_DE_ACHOCOLATADO_SEMIDESNATADO = estoqueInicialEmbalagensDeAchocolatadoSemidesnatado
    self.ESTOQUE_MINIMO_EMBALAGENS_DE_ACHOCOLATADO_SEMIDESNATADO = estoqueInicialEmbalagensDeAchocolatadoSemidesnatado * pontoDeRecompraDeEstoque
    self.estoqueEmbalagensDeAchocolatadoDesnatado = estoqueInicialEmbalagensDeAchocolatadoDesnatado
    self.ESTOQUE_MAXIMO_EMBALAGENS_DE_ACHOCOLATADO_DESNATADO = estoqueInicialEmbalagensDeAchocolatadoDesnatado
    self.ESTOQUE_MINIMO_EMBALAGENS_DE_ACHOCOLATADO_DESNATADO = estoqueInicialEmbalagensDeAchocolatadoDesnatado * pontoDeRecompraDeEstoque
    self.estoqueLeiteCru = estoqueInicialLeiteCru
    self.ESTOQUE_MAXIMO_LEITE_CRU = estoqueInicialLeiteCru
    self.ESTOQUE_MINIMO_LEITE_CRU = estoqueInicialLeiteCru * pontoDeRecompraDeEstoque
    self.estoqueAçucarEChocolate = estoqueInicialAçucarEChocolate
    self.ESTOQUE_MAXIMO_AÇUCAR_E_CHOCOLATE = estoqueInicialAçucarEChocolate
    self.ESTOQUE_MINIMO_AÇUCAR_E_CHOCOLATE = estoqueInicialAçucarEChocolate * pontoDeRecompraDeEstoque

    self.getTempoPasteurização = getTempoPasteurização
    self.LIMITE_MAQUINA_PASTEURIZADORA = LIMITE_MAQUINA_PASTEURIZADORA
    # self.getTempoLimparMaquinaPasteurização = getTempoLimparMaquinaPasteurização
    self.getTempoCentrifugaçãoSemidesnatado = getTempoCentrifugaçãoSemidesnatado
    self.getTempoCentrifugaçãoDesnatado = getTempoCentrifugaçãoDesnatado
    self.getTempoAdiçãoAçucarEChocolate = getTempoAdiçãoAçucarEChocolate
    self.getTempoHomogeneização = getTempoHomogeneização
    self.getTempoEnvaseLeite = getTempoEnvaseLeite
    self.getTempoEnvaseAchocolatado = getTempoEnvaseAchocolatado

    self.getTempoDeEntregaLeiteCruComprado = getTempoDeEntregaLeiteCruComprado

    self.MaquinaPasteurizadora = simpy.Resource(env, capacity=quantidadeMaquinasPasteurizadoras)
    self.MaquinaCentrífuga = simpy.Resource(env, capacity=quantidadeMaquinasCentrífugas)
    self.Container = simpy.Resource(env, capacity=quantidadeContainers)
    self.MaquinaEnvasadora = simpy.Resource(env, capacity=quantidadeMaquinasEnvasadoras)

    self.getTempoDeEsperaEntreClientes = getTempoDeEsperaEntreClientes
    self.demandaSemanalLitrosDeLeite = demandaSemanalLitrosDeLeite

    self.quantidadeVendidoProduto1 = 0
    self.quantidadeVendidoProduto2 = 0
    self.quantidadeVendidoProduto3 = 0
    self.quantidadeVendidoProduto4 = 0
    self.quantidadeVendidoProduto5 = 0
    self.quantidadeVendidoProduto6 = 0

    self.quantidadePedidos = 0
    self.quantidadePedidosNãoAtendidos = 0

    self.estaProduzindoLeite = False
    self.estaProduzindoAchocolatado = False
    self.estaComprandoLeiteCru = False
    self.estaComprandoAçucarEChocolate = False
    self.estaComprandoEmbalagemLeiteDesnatado = False
    self.estaComprandoEmbalagemLeiteSemidesnatado = False
    self.estaComprandoEmbalagemAchocolatadoDesnatado = False
    self.estaComprandoEmbalagemAchocolatadoSemidesnatado = False

    self.quantidadeDeVendas = 0
    self.tempoAtrasoDaVenda = 0
    self.quantidadeDeBateladas = 0
    self.quantidadeDeCompras = 0

    self.custo = 0

    self.custo += (70000 + 75*(LIMITE_MAQUINA_PASTEURIZADORA - 1000)) / (52*10) # Custo da máquina pasteurizadora com vida útil de 10 anos
    self.custo += (self.ESTOQUE_MAXIMO_LEITE_CRU * 12000/1000) / (52*10)

  def produzLeite(self, quantidadeLeiteDesnatado, quantidadeLeiteSemidesnatado):
    # print(f'Entrou precisando produzir {quantidadeLeiteDesnatado + quantidadeLeiteSemidesnatado}')
    if (self.env.now % (60*24) < 60*7):
      yield self.env.timeout(60*7 - self.env.now % (60*24))
    elif (self.env.now % (60*24) > 60*15):
      yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
    if (self.estoqueLeiteCru < quantidadeLeiteDesnatado + quantidadeLeiteSemidesnatado):
      self.env.process(self.compraLeiteCruEProduzLeite(quantidadeASerEncomendada=(self.ESTOQUE_MAXIMO_LEITE_CRU - self.estoqueLeiteCru), quantidadeLeiteDesnatado=quantidadeLeiteDesnatado, quantidadeLeiteSemidesnatado=quantidadeLeiteSemidesnatado))
      return
    self.estaProduzindoLeite = True
    _tempo = self.env.now
    # print("ALô, BOIADA")
    # print("ALô, BOIADA")
    # print("ALô, BOIADA")
    # print("ALô, BOIADA")
    while (quantidadeLeiteDesnatado + quantidadeLeiteSemidesnatado > self.LIMITE_MAQUINA_PASTEURIZADORA):
      # print(f'Looping: {quantidadeLeiteDesnatado + quantidadeLeiteSemidesnatado}')
      quantidadeLeiteDesnatado *= 0.9
      quantidadeLeiteSemidesnatado *= 0.9

    # if (self.env.now - _tempo > 2460): raise Exception("Fiquei esperando feito bobo")
    # print(f'O estoque do leite cru antes: {self.estoqueLeiteCru} | depois: {self.estoqueLeiteCru - (quantidadeLeiteSemidesnatado + quantidadeLeiteDesnatado)}, at {self.env.now}')
    if (self.estoqueLeiteCru < quantidadeLeiteDesnatado + quantidadeLeiteSemidesnatado):
      self.env.process(self.compraLeiteCruEProduzLeite(quantidadeASerEncomendada=(self.ESTOQUE_MAXIMO_LEITE_CRU - self.estoqueLeiteCru), quantidadeLeiteDesnatado=quantidadeLeiteDesnatado, quantidadeLeiteSemidesnatado=quantidadeLeiteSemidesnatado))
      self.estaProduzindoLeite = False
      return
    self.estoqueLeiteCru -= quantidadeLeiteSemidesnatado + quantidadeLeiteDesnatado

    if (self.estoqueLeiteCru < 0): raise Exception("COISAS DERAM MT ERRADO")

    if (self.estoqueLeiteCru < self.ESTOQUE_MINIMO_LEITE_CRU):
      self.env.process(self.compraLeiteCru(self.ESTOQUE_MAXIMO_LEITE_CRU - self.estoqueLeiteCru))

    _tempo = self.env.now

    with self.MaquinaPasteurizadora.request() as req:
      yield req
      yield self.env.timeout(self.getTempoPasteurização())

    # print(self.env.now - _tempo)

    with self.MaquinaCentrífuga.request() as req:
      yield req
      tempoCentrifugaçãoDesnatado, tempoCentrifugaçãoSemidesnatado = self.getTempoCentrifugaçãoDesnatado(), self.getTempoCentrifugaçãoSemidesnatado()
      if (quantidadeLeiteSemidesnatado > 0 and quantidadeLeiteDesnatado == 0):
        yield self.env.timeout(tempoCentrifugaçãoSemidesnatado)
      else:
        yield self.env.timeout(tempoCentrifugaçãoDesnatado)


    if (self.estaComprandoEmbalagemLeiteSemidesnatado == False and quantidadeLeiteSemidesnatado > self.estoqueEmbalagemDeSemidesnatado):
      self.env.process(self.compraEmbalagensSemidesnatado(quantidadeLeiteSemidesnatado - self.estoqueEmbalagemDeSemidesnatado))
    if (self.estaComprandoEmbalagemLeiteDesnatado == False and quantidadeLeiteDesnatado > self.estoqueEmbalagemDeDesnatado):
      self.env.process(self.compraEmbalagensDesnatado(quantidadeLeiteDesnatado - self.estoqueEmbalagemDeDesnatado))

    # while (quantidadeLeiteSemidesnatado > self.estoqueEmbalagemDeSemidesnatado):
    #   yield self.env.timeout(1) if self.estaComprandoEmbalagemLeiteSemidesnatado == True else self.env.process(self.compraEmbalagensSemidesnatado(quantidadeLeiteSemidesnatado - self.estoqueEmbalagemDeSemidesnatado))
    #   # if (self.env.now - _tempo > 2460): raise Exception("Fiquei esperando feito bobo")
    # while (quantidadeLeiteDesnatado > self.estoqueEmbalagemDeDesnatado):
    #   yield self.env.timeout(1) if self.estaComprandoEmbalagemLeiteDesnatado == True else self.env.process(self.compraEmbalagensDesnatado(quantidadeLeiteDesnatado - self.estoqueEmbalagemDeDesnatado))
    #   # if (self.env.now - _tempo > 2460*2): raise Exception("Fiquei esperando feito bobo")


    # print(f'O estoque de embalagem de desnatado antes: {self.estoqueEmbalagemDeDesnatado} | depois: {self.estoqueEmbalagemDeDesnatado - (quantidadeLeiteDesnatado)}, at {self.env.now}')

    if (self.estoqueEmbalagemDeDesnatado < quantidadeLeiteDesnatado):
      self.env.process(self.compraEmbalagensDesnatadoEProduzLeite(quantidadeASerEncomendada=(self.ESTOQUE_MAXIMO_EMBALAGEM_DE_DESNATADO - self.estoqueEmbalagemDeDesnatado), quantidadeLeiteDesnatado=quantidadeLeiteDesnatado, quantidadeLeiteSemidesnatado=quantidadeLeiteSemidesnatado))
      self.estaProduzindoLeite = False
      return
    self.estoqueEmbalagemDeDesnatado -= quantidadeLeiteDesnatado
    # print(f'O estoque de semidesnatado antes: {self.estoqueEmbalagemDeSemidesnatado} | depois: {self.estoqueEmbalagemDeSemidesnatado - (quantidadeLeiteSemidesnatado)}, at {self.env.now}')
    if (self.estoqueEmbalagemDeSemidesnatado < quantidadeLeiteSemidesnatado):
      self.env.process(self.compraEmbalagensSemidesnatadoEProduzLeite(quantidadeASerEncomendada=(self.ESTOQUE_MAXIMO_EMBALAGEM_DE_SEMIDESNATADO - self.estoqueEmbalagemDeSemidesnatado), quantidadeLeiteDesnatado=quantidadeLeiteDesnatado, quantidadeLeiteSemidesnatado=quantidadeLeiteSemidesnatado))
      self.estaProduzindoLeite = False
      return
    self.estoqueEmbalagemDeSemidesnatado -= quantidadeLeiteSemidesnatado
    if (self.estoqueEmbalagemDeDesnatado < self.ESTOQUE_MINIMO_EMBALAGEM_DE_DESNATADO):
      self.env.process(self.compraEmbalagensDesnatado(self.ESTOQUE_MAXIMO_EMBALAGEM_DE_DESNATADO - self.estoqueEmbalagemDeDesnatado))
    if (self.estoqueEmbalagemDeSemidesnatado < self.ESTOQUE_MINIMO_EMBALAGEM_DE_SEMIDESNATADO):
      self.env.process(self.compraEmbalagensSemidesnatado(self.ESTOQUE_MAXIMO_EMBALAGEM_DE_SEMIDESNATADO - self.estoqueEmbalagemDeSemidesnatado))

    with self.Container.request() as req:
      yield req
      with self.MaquinaEnvasadora.request() as req:
        yield req
        yield self.env.timeout(self.getTempoEnvaseLeite(quantidadeLeiteDesnatado + quantidadeLeiteSemidesnatado))
    # print(f'Estoque de leite desnatado embalado antes: {self.estoqueLeiteDesnatadoEmbalado} | Depois: {self.estoqueLeiteDesnatadoEmbalado + quantidadeLeiteDesnatado}, at {self.env.now}')
    self.estoqueLeiteDesnatadoEmbalado += quantidadeLeiteDesnatado
    # print(f'Estoque de leite desnatado embalado antes: {self.estoqueLeiteSemidesnatadoEmbalado} | Depois: {self.estoqueLeiteSemidesnatadoEmbalado + quantidadeLeiteSemidesnatado}, at {self.env.now}')
    self.estoqueLeiteSemidesnatadoEmbalado += quantidadeLeiteSemidesnatado
    # print(quantidadeLeiteDesnatado + quantidadeLeiteSemidesnatado)
    #Aqui
    self.estaProduzindoLeite = False
    self.quantidadeDeBateladas += 1

    if (self.estoqueLeiteDesnatadoEmbalado < self.ESTOQUE_MINIMO_LEITE_DESNATADO_EMBALADO or self.estoqueLeiteSemidesnatadoEmbalado < self.ESTOQUE_MINIMO_LEITE_SEMIDESNATADO_EMBALADO):
      self.env.process(self.produzLeite(quantidadeLeiteDesnatado=np.max([0, self.ESTOQUE_MAXIMO_LEITE_DESNATADO_EMBALADO - self.estoqueLeiteDesnatadoEmbalado]), quantidadeLeiteSemidesnatado=np.max([0, self.ESTOQUE_MAXIMO_LEITE_SEMIDESNATADO_EMBALADO - self.estoqueLeiteSemidesnatadoEmbalado])))

  def produzAchocolatado(self, quantidadeAchocolatadoDesnatado, quantidadeAchocolatadoSemidesnatado):
    if (self.env.now % (60*24) < 60*7):
      yield self.env.timeout(60*7 - self.env.now % (60*24))
    elif (self.env.now % (60*24) > 60*15):
      yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
    if (self.estoqueLeiteCru < quantidadeAchocolatadoDesnatado + quantidadeAchocolatadoSemidesnatado):
      self.env.process(self.compraLeiteCruEProduzLeite(quantidadeASerEncomendada=(self.ESTOQUE_MAXIMO_LEITE_CRU - self.estoqueLeiteCru), quantidadeLeiteDesnatado=quantidadeAchocolatadoDesnatado, quantidadeLeiteSemidesnatado=quantidadeAchocolatadoSemidesnatado))
      return
    self.estaProduzindoAchocolatado = True
    # print("ALÔ BOIZINHO")
    # print("ALÔ BOIZINHO")
    # print("ALÔ BOIZINHO")
    # print("ALÔ BOIZINHO")
    while (quantidadeAchocolatadoDesnatado + quantidadeAchocolatadoSemidesnatado > self.LIMITE_MAQUINA_PASTEURIZADORA):
      quantidadeAchocolatadoDesnatado *= 0.9
      quantidadeAchocolatadoSemidesnatado *= 0.9

    # print(f'O estoque do leite cru antes: {self.estoqueLeiteCru} | depois: {self.estoqueLeiteCru - (quantidadeAchocolatadoDesnatado + quantidadeAchocolatadoSemidesnatado)}, at {self.env.now}')
    self.estoqueLeiteCru -= quantidadeAchocolatadoDesnatado + quantidadeAchocolatadoSemidesnatado

    if (self.estoqueLeiteCru < self.ESTOQUE_MINIMO_LEITE_CRU):
      self.env.process(self.compraLeiteCru(self.ESTOQUE_MAXIMO_LEITE_CRU - self.estoqueLeiteCru))

    with self.MaquinaPasteurizadora.request() as req:
      yield req
      yield self.env.timeout(self.getTempoPasteurização())

    with self.MaquinaCentrífuga.request() as req:
      yield req
      tempoCentrifugaçãoDesnatado, tempoCentrifugaçãoSemidesnatado = self.getTempoCentrifugaçãoDesnatado(), self.getTempoCentrifugaçãoSemidesnatado()
      if (quantidadeAchocolatadoSemidesnatado > 0 and quantidadeAchocolatadoDesnatado == 0):
        yield self.env.timeout(tempoCentrifugaçãoSemidesnatado)
      else:
        yield self.env.timeout(tempoCentrifugaçãoDesnatado)

    _tempo = self.env.now

    if (self.estaComprandoAçucarEChocolate == False and self.estoqueAçucarEChocolate < (quantidadeAchocolatadoDesnatado + quantidadeAchocolatadoSemidesnatado)*5):
      self.env.process(self.compraAçucarEChocolate((quantidadeAchocolatadoDesnatado + quantidadeAchocolatadoSemidesnatado)*5 - self.estoqueAçucarEChocolate))
    if (self.estaComprandoEmbalagemAchocolatadoSemidesnatado == False and quantidadeAchocolatadoSemidesnatado * 5 > self.estoqueEmbalagensDeAchocolatadoSemidesnatado):
      self.env.process(self.compraEmbalagensAchocolatadoSemidesnatado(quantidadeAchocolatadoSemidesnatado * 5 - self.estoqueEmbalagensDeAchocolatadoSemidesnatado))
    if (self.estaComprandoEmbalagemAchocolatadoDesnatado == False and quantidadeAchocolatadoDesnatado * 5 > self.estoqueEmbalagensDeAchocolatadoDesnatado):
      self.env.process(self.compraEmbalagensAchocolatadoDesnatado(quantidadeAchocolatadoDesnatado * 5 - self.estoqueEmbalagensDeAchocolatadoDesnatado))

    # while (self.estoqueAçucarEChocolate < (quantidadeAchocolatadoDesnatado + quantidadeAchocolatadoSemidesnatado)*5):
    #   yield self.env.timeout(1) if self.estaComprandoAçucarEChocolate == True else self.env.process(self.compraAçucarEChocolate((quantidadeAchocolatadoDesnatado + quantidadeAchocolatadoSemidesnatado)*5 - self.estoqueAçucarEChocolate))
      # if (self.env.now - _tempo > 2460): raise Exception("Fiquei esperando feito bobo")

    # print(f'1O estoque de açucar e chocolate antes: {self.estoqueAçucarEChocolate} | depois: {self.estoqueAçucarEChocolate - (quantidadeAchocolatadoDesnatado + quantidadeAchocolatadoSemidesnatado)*5}, at {self.env.now}')
    if (self.estoqueAçucarEChocolate < (quantidadeAchocolatadoDesnatado + quantidadeAchocolatadoSemidesnatado)*5):
      self.estaProduzindoAchocolatado = False
      self.env.process(self.compraAçucarEChocolateEProduzAchocolatado(quantidadeASerEncomendada=(self.ESTOQUE_MAXIMO_AÇUCAR_E_CHOCOLATE - self.estoqueAçucarEChocolate), quantidadeAchocolatadoDesnatado=quantidadeAchocolatadoDesnatado, quantidadeAchocolatadoSemidesnatado=quantidadeAchocolatadoSemidesnatado))
      return
    if (self.estoqueAçucarEChocolate < (quantidadeAchocolatadoDesnatado + quantidadeAchocolatadoSemidesnatado)*5):
      raise Exception("Eu sou um pão")
    self.estoqueAçucarEChocolate -= (quantidadeAchocolatadoDesnatado + quantidadeAchocolatadoSemidesnatado)*5

    if (self.estoqueAçucarEChocolate < self.ESTOQUE_MINIMO_AÇUCAR_E_CHOCOLATE):
      self.env.process(self.compraAçucarEChocolate(self.ESTOQUE_MAXIMO_AÇUCAR_E_CHOCOLATE - self.estoqueAçucarEChocolate))



    # print(f'O estoque de embalagens de achocolatado desnatado antes: {self.estoqueEmbalagensDeAchocolatadoDesnatado} | depois: {self.estoqueEmbalagensDeAchocolatadoDesnatado - (quantidadeAchocolatadoDesnatado * 5)}, at {self.env.now}')
    if (self.estoqueEmbalagensDeAchocolatadoDesnatado < quantidadeAchocolatadoDesnatado * 5):
      self.env.process(self.compraEmbalagensAchocolatadoDesnatadoEProduzAchocolatado(quantidadeASerEncomendada=(self.ESTOQUE_MAXIMO_EMBALAGENS_DE_ACHOCOLATADO_DESNATADO - self.estoqueEmbalagensDeAchocolatadoDesnatado), quantidadeAchocolatadoDesnatado=quantidadeAchocolatadoDesnatado, quantidadeAchocolatadoSemidesnatado=quantidadeAchocolatadoSemidesnatado))
      self.estaProduzindoAchocolatado = False
      return
    self.estoqueEmbalagensDeAchocolatadoDesnatado -= quantidadeAchocolatadoDesnatado * 5
    # print(f'O estoque de embalagens de achocolatado semidesnatado antes: {self.estoqueEmbalagensDeAchocolatadoSemidesnatado} | depois: {self.estoqueEmbalagensDeAchocolatadoSemidesnatado - (quantidadeAchocolatadoSemidesnatado * 5)}, at {self.env.now}')
    if (self.estoqueEmbalagensDeAchocolatadoSemidesnatado < quantidadeAchocolatadoSemidesnatado * 5):
      self.env.process(self.compraEmbalagensAchocolatadoSemidesnatadoEProduzAchocolatado(quantidadeASerEncomendada=(self.ESTOQUE_MAXIMO_EMBALAGENS_DE_ACHOCOLATADO_SEMIDESNATADO - self.estoqueEmbalagensDeAchocolatadoSemidesnatado), quantidadeAchocolatadoDesnatado=quantidadeAchocolatadoDesnatado, quantidadeAchocolatadoSemidesnatado=quantidadeAchocolatadoSemidesnatado))
      self.estaProduzindoAchocolatado = False
      return
    self.estoqueEmbalagensDeAchocolatadoSemidesnatado -= quantidadeAchocolatadoSemidesnatado * 5
    if (self.estoqueEmbalagensDeAchocolatadoDesnatado < self.ESTOQUE_MINIMO_EMBALAGENS_DE_ACHOCOLATADO_DESNATADO):
      self.env.process(self.compraEmbalagensAchocolatadoDesnatado(self.ESTOQUE_MAXIMO_EMBALAGENS_DE_ACHOCOLATADO_DESNATADO - self.estoqueEmbalagensDeAchocolatadoDesnatado))
    if (self.estoqueEmbalagensDeAchocolatadoSemidesnatado < self.ESTOQUE_MINIMO_EMBALAGENS_DE_ACHOCOLATADO_SEMIDESNATADO):
      self.env.process(self.compraEmbalagensAchocolatadoSemidesnatado(self.ESTOQUE_MAXIMO_EMBALAGENS_DE_ACHOCOLATADO_SEMIDESNATADO - self.estoqueEmbalagensDeAchocolatadoSemidesnatado))

    with self.Container.request() as req:
      yield req
      with self.MaquinaEnvasadora.request() as req:
        yield req
        yield self.env.timeout(self.getTempoEnvaseAchocolatado(quantidadeAchocolatadoDesnatado + quantidadeAchocolatadoSemidesnatado))
    # print(f'Estoque de achocolatado desnatado embalado antes: {self.estoqueAchocolatadoDesnatadoEmbalado} | depois: {self.estoqueAchocolatadoDesnatadoEmbalado + quantidadeAchocolatadoDesnatado}, at {self.env.now}')
    self.estoqueAchocolatadoDesnatadoEmbalado += quantidadeAchocolatadoDesnatado
    # print(f'Estoque de achocolatado desnatado embalado antes: {self.estoqueAchocolatadoSemidesnatadoEmbalado} | depois: {self.estoqueAchocolatadoSemidesnatadoEmbalado + quantidadeAchocolatadoSemidesnatado}, at {self.env.now}')
    self.estoqueAchocolatadoSemidesnatadoEmbalado += quantidadeAchocolatadoSemidesnatado
    self.estaProduzindoAchocolatado = False
    self.quantidadeDeBateladas += 1

    if (self.estoqueAchocolatadoDesnatadoEmbalado < self.ESTOQUE_MINIMO_ACHOCOLATADO_DESNATADO_EMBALADO or self.estoqueAchocolatadoSemidesnatadoEmbalado < self.ESTOQUE_MINIMO_ACHOCOLATADO_SEMIDESNATADO_EMBALADO):
      self.env.process(self.produzAchocolatado(quantidadeAchocolatadoDesnatado=np.max([0.0,self.ESTOQUE_MAXIMO_ACHOCOLATADO_DESNATADO_EMBALADO - self.estoqueAchocolatadoDesnatadoEmbalado]), quantidadeAchocolatadoSemidesnatado=np.max([0.0,self.ESTOQUE_MAXIMO_ACHOCOLATADO_SEMIDESNATADO_EMBALADO - self.estoqueAchocolatadoSemidesnatadoEmbalado])))


  def compraLeiteCru(self, quantidadeASerEncomendada):
    self.estaComprandoLeiteCru = True
    #Aqui estamos pressupondo que haja sempre uma fazenda que recebe pedidos das 07h até as 15h
    if (self.env.now % (60*24) < 60*7):
      yield self.env.timeout(60*7 - self.env.now % (60*24))
    elif (self.env.now % (60*24) > 60*15):
      yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
    else:
      yield self.env.timeout(self.getTempoDeEntregaLeiteCruComprado())
    # print(f'O estoque de leite cru antes: {self.estoqueLeiteCru} | depois: {self.estoqueLeiteCru + quantidadeASerEncomendada}, at {self.env.now}')
    self.estoqueLeiteCru += quantidadeASerEncomendada
    self.estaComprandoLeiteCru = False
    self.quantidadeDeCompras += 1

  def compraLeiteCruEProduzLeite(self, quantidadeASerEncomendada, quantidadeLeiteDesnatado, quantidadeLeiteSemidesnatado):
    self.estaComprandoLeiteCru = True
    #Aqui estamos pressupondo que haja sempre uma fazenda que recebe pedidos das 07h até as 15h
    if (self.env.now % (60*24) < 60*7):
      yield self.env.timeout(60*7 - self.env.now % (60*24))
    elif (self.env.now % (60*24) > 60*15):
      yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
    else:
      yield self.env.timeout(self.getTempoDeEntregaLeiteCruComprado())
    # print(f'O estoque de leite cru antes: {self.estoqueLeiteCru} | depois: {self.estoqueLeiteCru + quantidadeASerEncomendada}, at {self.env.now}')
    self.estoqueLeiteCru += quantidadeASerEncomendada
    self.estaComprandoLeiteCru = False
    self.quantidadeDeCompras += 1
    self.env.process(self.produzLeite(quantidadeLeiteDesnatado=quantidadeLeiteDesnatado, quantidadeLeiteSemidesnatado=quantidadeLeiteSemidesnatado))

  def compraLeiteCruEProduzAchocolatado(self, quantidadeASerEncomendada, quantidadeLeiteDesnatado, quantidadeLeiteSemidesnatado):
    self.estaComprandoLeiteCru = True
    #Aqui estamos pressupondo que haja sempre uma fazenda que recebe pedidos das 07h até as 15h
    if (self.env.now % (60*24) < 60*7):
      yield self.env.timeout(60*7 - self.env.now % (60*24))
    elif (self.env.now % (60*24) > 60*15):
      yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
    else:
      yield self.env.timeout(self.getTempoDeEntregaLeiteCruComprado())
    # print(f'O estoque de leite cru antes: {self.estoqueLeiteCru} | depois: {self.estoqueLeiteCru + quantidadeASerEncomendada}, at {self.env.now}')
    self.estoqueLeiteCru += quantidadeASerEncomendada
    self.estaComprandoLeiteCru = False
    self.quantidadeDeCompras += 1
    self.env.process(self.produzAchocolatado(quantidadeAchocolatadoDesnatado=quantidadeLeiteDesnatado, quantidadeAchocolatadoSemidesnatado=quantidadeLeiteSemidesnatado))


  def compraEmbalagensDesnatado(self, quantidadeASerEncomendada):
    self.estaComprandoEmbalagemLeiteDesnatado = True
    #Aqui estamos pressupondo que haja sempre uma fábrica que recebe pedidos a qualquer momento e entrega às 07h do dia seguinte
    yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
    #print(f'O Estoque de embalagem de leite desnatado antes: {self.estoqueEmbalagemDeDesnatado} | depois: {self.estoqueEmbalagemDeDesnatado + quantidadeASerEncomendada}, at {self.env.now}')
    self.estoqueEmbalagemDeDesnatado += quantidadeASerEncomendada
    self.estaComprandoEmbalagemLeiteDesnatado = False
    self.quantidadeDeCompras += 1

  def compraEmbalagensDesnatadoEProduzLeite(self, quantidadeASerEncomendada, quantidadeLeiteDesnatado, quantidadeLeiteSemidesnatado):
    self.estaComprandoEmbalagemLeiteDesnatado = True
    #Aqui estamos pressupondo que haja sempre uma fábrica que recebe pedidos a qualquer momento e entrega às 07h do dia seguinte
    yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
    #print(f'O Estoque de embalagem de leite desnatado antes: {self.estoqueEmbalagemDeDesnatado} | depois: {self.estoqueEmbalagemDeDesnatado + quantidadeASerEncomendada}, at {self.env.now}')
    self.estoqueEmbalagemDeDesnatado += quantidadeASerEncomendada
    self.estaComprandoEmbalagemLeiteDesnatado = False
    self.quantidadeDeCompras += 1
    self.env.process(self.produzLeite(quantidadeLeiteDesnatado=quantidadeLeiteDesnatado, quantidadeLeiteSemidesnatado=quantidadeLeiteSemidesnatado))


  def compraEmbalagensSemidesnatado(self, quantidadeASerEncomendada):
    self.estaComprandoEmbalagemLeiteSemidesnatado = True
    #Aqui estamos pressupondo que haja sempre uma fábrica que recebe pedidos a qualquer momento e entrega às 07h do dia seguinte
    yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
    #print(f'O Estoque de embalagem de leite semidesnatado antes: {self.estoqueEmbalagemDeSemidesnatado} | depois: {self.estoqueEmbalagemDeSemidesnatado + quantidadeASerEncomendada}, at {self.env.now}')
    self.estoqueEmbalagemDeSemidesnatado += quantidadeASerEncomendada
    self.estaComprandoEmbalagemLeiteSemidesnatado = False
    self.quantidadeDeCompras += 1

  def compraEmbalagensSemidesnatadoEProduzLeite(self, quantidadeASerEncomendada, quantidadeLeiteDesnatado, quantidadeLeiteSemidesnatado):
    self.estaComprandoEmbalagemLeiteSemidesnatado = True
    #Aqui estamos pressupondo que haja sempre uma fábrica que recebe pedidos a qualquer momento e entrega às 07h do dia seguinte
    yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
    #print(f'O Estoque de embalagem de leite semidesnatado antes: {self.estoqueEmbalagemDeSemidesnatado} | depois: {self.estoqueEmbalagemDeSemidesnatado + quantidadeASerEncomendada}, at {self.env.now}')
    self.estoqueEmbalagemDeSemidesnatado += quantidadeASerEncomendada
    self.estaComprandoEmbalagemLeiteSemidesnatado = False
    self.quantidadeDeCompras += 1
    self.env.process(self.produzLeite(quantidadeLeiteDesnatado=quantidadeLeiteDesnatado, quantidadeLeiteSemidesnatado=quantidadeLeiteSemidesnatado))


  def compraEmbalagensAchocolatadoDesnatado(self, quantidadeASerEncomendada):
    self.estaComprandoEmbalagemAchocolatadoDesnatado = True
    #Aqui estamos pressupondo que haja sempre uma fábrica que recebe pedidos a qualquer momento e entrega às 07h do dia seguinte
    yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
    #print(f'O Estoque de embalagem de achocolatado desnatado antes: {self.estoqueEmbalagensDeAchocolatadoDesnatado} | depois: {self.estoqueEmbalagensDeAchocolatadoDesnatado + quantidadeASerEncomendada}, at {self.env.now}')
    self.estoqueEmbalagensDeAchocolatadoDesnatado += quantidadeASerEncomendada
    self.estaComprandoEmbalagemAchocolatadoDesnatado = False
    self.quantidadeDeCompras += 1

  def compraEmbalagensAchocolatadoDesnatadoEProduzAchocolatado(self, quantidadeASerEncomendada, quantidadeAchocolatadoDesnatado, quantidadeAchocolatadoSemidesnatado):
    self.estaComprandoEmbalagemAchocolatadoDesnatado = True
    #Aqui estamos pressupondo que haja sempre uma fábrica que recebe pedidos a qualquer momento e entrega às 07h do dia seguinte
    yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
    #print(f'O Estoque de embalagem de achocolatado desnatado antes: {self.estoqueEmbalagensDeAchocolatadoDesnatado} | depois: {self.estoqueEmbalagensDeAchocolatadoDesnatado + quantidadeASerEncomendada}, at {self.env.now}')
    self.estoqueEmbalagensDeAchocolatadoDesnatado += quantidadeASerEncomendada
    self.estaComprandoEmbalagemAchocolatadoDesnatado = False
    self.quantidadeDeCompras += 1
    self.env.process(self.produzAchocolatado(quantidadeAchocolatadoDesnatado=quantidadeAchocolatadoDesnatado, quantidadeAchocolatadoSemidesnatado=quantidadeAchocolatadoSemidesnatado))


  def compraEmbalagensAchocolatadoSemidesnatado(self, quantidadeASerEncomendada):
    self.estaComprandoEmbalagemAchocolatadoSemidesnatado = True
    #Aqui estamos pressupondo que haja sempre uma fábrica que recebe pedidos a qualquer momento e entrega às 07h do dia seguinte
    yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
    #print(f'O Estoque de embalagem de achocolatado semidesnatado antes: {self.estoqueEmbalagensDeAchocolatadoSemidesnatado} | depois: {self.estoqueEmbalagensDeAchocolatadoSemidesnatado + quantidadeASerEncomendada}, at {self.env.now}')
    self.estoqueEmbalagensDeAchocolatadoSemidesnatado += quantidadeASerEncomendada
    self.estaComprandoEmbalagemAchocolatadoSemidesnatado = False
    self.quantidadeDeCompras += 1

  def compraEmbalagensAchocolatadoSemidesnatadoEProduzAchocolatado(self, quantidadeASerEncomendada, quantidadeAchocolatadoDesnatado, quantidadeAchocolatadoSemidesnatado):
    self.estaComprandoEmbalagemAchocolatadoSemidesnatado = True
    #Aqui estamos pressupondo que haja sempre uma fábrica que recebe pedidos a qualquer momento e entrega às 07h do dia seguinte
    yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
    #print(f'O Estoque de embalagem de achocolatado semidesnatado antes: {self.estoqueEmbalagensDeAchocolatadoSemidesnatado} | depois: {self.estoqueEmbalagensDeAchocolatadoSemidesnatado + quantidadeASerEncomendada}, at {self.env.now}')
    self.estoqueEmbalagensDeAchocolatadoSemidesnatado += quantidadeASerEncomendada
    self.estaComprandoEmbalagemAchocolatadoSemidesnatado = False
    self.quantidadeDeCompras += 1
    self.env.process(self.produzAchocolatado(quantidadeAchocolatadoDesnatado=quantidadeAchocolatadoDesnatado, quantidadeAchocolatadoSemidesnatado=quantidadeAchocolatadoSemidesnatado))


  def compraAçucarEChocolate(self, quantidadeASerEncomendada):
     self.estaComprandoAçucarEChocolate = True
     #Aqui estamos pressupondo que haja sempre uma fábrica que recebe pedidos a qualquer momento e entrega às 07h do dia seguinte
     yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
     #print(f'O Estoque de açucar e chocolate antes: {self.estoqueAçucarEChocolate} | depois: {self.estoqueAçucarEChocolate + quantidadeASerEncomendada}, at {self.env.now}')
     self.estoqueAçucarEChocolate += quantidadeASerEncomendada
     self.estaComprandoAçucarEChocolate = False
     self.quantidadeDeCompras += 1

  def compraAçucarEChocolateEProduzAchocolatado(self, quantidadeASerEncomendada, quantidadeAchocolatadoDesnatado, quantidadeAchocolatadoSemidesnatado):
     self.estaComprandoAçucarEChocolate = True
     #Aqui estamos pressupondo que haja sempre uma fábrica que recebe pedidos a qualquer momento e entrega às 07h do dia seguinte
     yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
     #print(f'O Estoque de açucar e chocolate antes: {self.estoqueAçucarEChocolate} | depois: {self.estoqueAçucarEChocolate + quantidadeASerEncomendada}, at {self.env.now}')
     self.estoqueAçucarEChocolate += quantidadeASerEncomendada
     self.estaComprandoAçucarEChocolate = False
     self.env.process(self.produzAchocolatado(quantidadeAchocolatadoDesnatado=quantidadeAchocolatadoDesnatado, quantidadeAchocolatadoSemidesnatado=quantidadeAchocolatadoSemidesnatado))
     self.quantidadeDeCompras += 1


  def cumpreComPedido(self, quantProduto1, quantProduto2, quantProduto3, quantProduto4, quantProduto5, quantProduto6):
    _tempo = self.env.now
    # if (self.estoqueLeiteDesnatadoEmbalado < quantProduto1*24.0 + quantProduto3*12.0 or self.estoqueLeiteSemidesnatadoEmbalado < quantProduto2*24.0 + quantProduto3*12.0):
    #   self.env.process(self.produzLeite(quantidadeLeiteDesnatado=np.max([0,self.ESTOQUE_MAXIMO_LEITE_DESNATADO_EMBALADO - self.estoqueLeiteDesnatadoEmbalado, 24*quantProduto1 + 12*quantProduto2 - self.estoqueLeiteDesnatadoEmbalado]), quantidadeLeiteSemidesnatado=np.max([0, self.ESTOQUE_MAXIMO_LEITE_SEMIDESNATADO_EMBALADO - self.estoqueLeiteSemidesnatadoEmbalado, quantProduto2*24 + 12*quantProduto3 - self.estoqueLeiteSemidesnatadoEmbalado])))
    # if (self.estoqueAchocolatadoDesnatadoEmbalado < quantProduto4*120.0 + quantProduto6*60.0 or self.estoqueAchocolatadoSemidesnatadoEmbalado < quantProduto5*120.0 + quantProduto6*60.0):
    #   self.env.process(self.produzAchocolatado(quantidadeAchocolatadoDesnatado=np.max([0,self.ESTOQUE_MAXIMO_ACHOCOLATADO_DESNATADO_EMBALADO - self.estoqueAchocolatadoDesnatadoEmbalado]), quantidadeAchocolatadoSemidesnatado=np.max([0, self.ESTOQUE_MAXIMO_ACHOCOLATADO_SEMIDESNATADO_EMBALADO - self.estoqueAchocolatadoSemidesnatadoEmbalado])))

    # print(quantProduto1, quantProduto2, quantProduto3, quantProduto4, quantProduto5, quantProduto6)
    # print(f'Estoque leite desnatado embalado agora: {self.estoqueLeiteDesnatadoEmbalado} | depois: {self.estoqueLeiteDesnatadoEmbalado - (quantProduto1*24.0 + quantProduto3*12.0)}, at {self.env.now}')
    if (self.estoqueLeiteDesnatadoEmbalado < quantProduto1*24.0 + quantProduto3*12.0):
      raise Exception("O verificador de estoque de leite desnatado embalado não funcionou")
    self.estoqueLeiteDesnatadoEmbalado -= quantProduto1*24.0 + quantProduto3*12.0
    if (self.estoqueLeiteDesnatadoEmbalado < self.ESTOQUE_MINIMO_LEITE_DESNATADO_EMBALADO and self.estaProduzindoLeite == False):
      self.env.process(self.produzLeite(quantidadeLeiteDesnatado=np.max([0,self.ESTOQUE_MAXIMO_LEITE_DESNATADO_EMBALADO - self.estoqueLeiteDesnatadoEmbalado]), quantidadeLeiteSemidesnatado=np.max([0, self.ESTOQUE_MAXIMO_LEITE_SEMIDESNATADO_EMBALADO - self.estoqueLeiteSemidesnatadoEmbalado])))


    if (self.estoqueLeiteSemidesnatadoEmbalado < quantProduto2*24.0 + quantProduto3*12.0):
      raise Exception("O verificador de estoque de leite semidesnatado embalado não funcionou")
    self.estoqueLeiteSemidesnatadoEmbalado -= quantProduto2*24.0 + quantProduto3*12.0
    if (self.estoqueLeiteSemidesnatadoEmbalado < self.ESTOQUE_MINIMO_LEITE_SEMIDESNATADO_EMBALADO and self.estaProduzindoLeite == False):
      self.env.process(self.produzLeite(quantidadeLeiteDesnatado=np.max([0,self.ESTOQUE_MAXIMO_LEITE_DESNATADO_EMBALADO - self.estoqueLeiteDesnatadoEmbalado]), quantidadeLeiteSemidesnatado=np.max([0, self.ESTOQUE_MAXIMO_LEITE_SEMIDESNATADO_EMBALADO - self.estoqueLeiteSemidesnatadoEmbalado])))


    # print(f'Estoque leite semidesnatado embalado agora: {self.estoqueLeiteSemidesnatadoEmbalado} | depois: {self.estoqueLeiteSemidesnatadoEmbalado - (quantProduto2*24.0 + quantProduto3*12.0)}, at {self.env.now}')
    # print(f'Estoque achocolatado desnatado embalado agora: {self.estoqueAchocolatadoDesnatadoEmbalado} | depois: {self.estoqueAchocolatadoDesnatadoEmbalado - (quantProduto4*120.0 + quantProduto6*60.0)}, at {self.env.now}')
    if (self.estoqueAchocolatadoDesnatadoEmbalado < quantProduto4*120.0 + quantProduto6*60.0):
      raise Exception("O verificador de estoque de achocolatado desnatado embalado não funcionou")
    self.estoqueAchocolatadoDesnatadoEmbalado -= quantProduto4*120.0 + quantProduto6*60.0
    if (self.estoqueAchocolatadoDesnatadoEmbalado < self.ESTOQUE_MINIMO_ACHOCOLATADO_DESNATADO_EMBALADO and self.estaProduzindoAchocolatado == False):
      self.env.process(self.produzAchocolatado(quantidadeAchocolatadoDesnatado=np.max([0.0,self.ESTOQUE_MAXIMO_ACHOCOLATADO_DESNATADO_EMBALADO - self.estoqueAchocolatadoDesnatadoEmbalado]), quantidadeAchocolatadoSemidesnatado=np.max([0.0,self.ESTOQUE_MAXIMO_ACHOCOLATADO_SEMIDESNATADO_EMBALADO - self.estoqueAchocolatadoSemidesnatadoEmbalado])))


    # print(f'Estoque achocolatado semidesnatado embalado agora: {self.estoqueAchocolatadoSemidesnatadoEmbalado} | depois: {self.estoqueAchocolatadoSemidesnatadoEmbalado - (quantProduto5*120.0 + quantProduto6*60.0)}, at {self.env.now}')
    if (self.estoqueAchocolatadoSemidesnatadoEmbalado < quantProduto5*120.0 + quantProduto6*60.0):
      raise Exception("O verificador de estoque de achocolatado semidesnatado embalado não funcionou")
    self.estoqueAchocolatadoSemidesnatadoEmbalado -= quantProduto5*120.0 + quantProduto6*60.0
    if (self.estoqueAchocolatadoSemidesnatadoEmbalado < self.ESTOQUE_MINIMO_ACHOCOLATADO_SEMIDESNATADO_EMBALADO and self.estaProduzindoAchocolatado == False):
      self.env.process(self.produzAchocolatado(quantidadeAchocolatadoDesnatado=np.max([0.0,self.ESTOQUE_MAXIMO_ACHOCOLATADO_DESNATADO_EMBALADO - self.estoqueAchocolatadoDesnatadoEmbalado]), quantidadeAchocolatadoSemidesnatado=np.max([0.0,self.ESTOQUE_MAXIMO_ACHOCOLATADO_SEMIDESNATADO_EMBALADO - self.estoqueAchocolatadoSemidesnatadoEmbalado])))

    self.quantidadeVendidoProduto1 += quantProduto1
    self.quantidadeVendidoProduto2 += quantProduto2
    self.quantidadeVendidoProduto3 += quantProduto3
    self.quantidadeVendidoProduto4 += quantProduto4
    self.quantidadeVendidoProduto5 += quantProduto5
    self.quantidadeVendidoProduto6 += quantProduto6

    self.custo += 24*1.8*0.2*(quantProduto1 + quantProduto2 + quantProduto3) #24 litros * 2.5 reais/litro * 0.2 (fator de custo do estoque)
    self.custo += 24*2*0.1*(quantProduto4 + quantProduto5 + quantProduto6)

    self.quantidadeDeVendas += 1

    # if (self.estaProduzindoLeite == False and (self.estoqueLeiteDesnatadoEmbalado < self.ESTOQUE_MINIMO_LEITE_DESNATADO_EMBALADO or self.estoqueLeiteSemidesnatadoEmbalado < self.ESTOQUE_MINIMO_LEITE_SEMIDESNATADO_EMBALADO)):
    #   self.env.process(self.produzLeite(quantidadeLeiteDesnatado=np.max([0, self.ESTOQUE_MAXIMO_LEITE_DESNATADO_EMBALADO - self.estoqueLeiteDesnatadoEmbalado]), quantidadeLeiteSemidesnatado=np.max([0, self.ESTOQUE_MAXIMO_LEITE_SEMIDESNATADO_EMBALADO - self.estoqueLeiteSemidesnatadoEmbalado])))

    # if (self.estaProduzindoAchocolatado == False and (self.estoqueAchocolatadoDesnatadoEmbalado < self.ESTOQUE_MINIMO_ACHOCOLATADO_DESNATADO_EMBALADO or self.estoqueAchocolatadoSemidesnatadoEmbalado < self.ESTOQUE_MINIMO_ACHOCOLATADO_SEMIDESNATADO_EMBALADO)):
    #   self.env.process(self.produzAchocolatado(quantidadeAchocolatadoDesnatado=np.max([0.0,self.ESTOQUE_MAXIMO_ACHOCOLATADO_DESNATADO_EMBALADO - self.estoqueAchocolatadoDesnatadoEmbalado]), quantidadeAchocolatadoSemidesnatado=np.max([0.0,self.ESTOQUE_MAXIMO_ACHOCOLATADO_SEMIDESNATADO_EMBALADO - self.estoqueAchocolatadoSemidesnatadoEmbalado])))


  def run(self):
    while True:
      # if (
      #     self.estoqueAchocolatadoDesnatadoEmbalado < 0 or
      #     self.estoqueAchocolatadoSemidesnatadoEmbalado < 0 or
      #     self.estoqueAçucarEChocolate < 0 or
      #     self.estoqueEmbalagemDeDesnatado < 0 or
      #     self.estoqueEmbalagemDeSemidesnatado < 0 or
      #     self.estoqueEmbalagensDeAchocolatadoDesnatado < 0 or
      #     self.estoqueEmbalagensDeAchocolatadoSemidesnatado < 0 or
      #     self.estoqueLeiteCru < 0 or
      #     self.estoqueLeiteDesnatadoEmbalado < 0 or
      #     self.estoqueLeiteSemidesnatadoEmbalado < 0
      # ):
      #     print()
      #     print()
      #     print()
      #     print()
      #     print(self.estoqueAchocolatadoDesnatadoEmbalado)
      #     print(self.estoqueAchocolatadoSemidesnatadoEmbalado)
      #     print(self.estoqueAçucarEChocolate)
      #     print(self.estoqueEmbalagemDeDesnatado)
      #     print(self.estoqueEmbalagemDeSemidesnatado)
      #     print(self.estoqueEmbalagensDeAchocolatadoDesnatado)
      #     print(self.estoqueEmbalagensDeAchocolatadoSemidesnatado)
      #     print(self.estoqueLeiteCru)
      #     print(self.estoqueLeiteDesnatadoEmbalado)
      #     print(self.estoqueLeiteSemidesnatadoEmbalado)
      #     raise Exception("Algum estoque está negativo KEKW")


      if (self.env.now % (60*24) < 60*7):
        yield self.env.timeout(60*7 - self.env.now % (60*24))
        yield self.env.timeout(np.random.exponential(scale=60))
      elif (self.env.now % (60*24) > 60*15):
        yield self.env.timeout(60*7 + 60*24 - self.env.now % (60*24))
        yield self.env.timeout(np.random.exponential(scale=60))
      else:
        yield self.env.timeout(np.random.exponential(scale=60))
      _ = ((self.demandaSemanalLitrosDeLeite / (5*8)) / 24)

      _demand1 = np.random.normal(_*1.5/4.0625, (_*1.5/4.0625) / 10)
      _demand2 = np.random.normal(_*1/4.0625, (_*1/4.0625) / 10)
      _demand3 = np.random.normal(_*.75/4.0625, (_*.75/4.0625) / 10)
      _demand4 = np.random.normal(_*.375/4.0625, (_*.375/4.0625) / 10)
      _demand5 = np.random.normal(_*.25/4.0625, (_*.25/4.0625) / 10)
      _demand6 = np.random.normal(_*.1875/4.0625, (_*.1875/4.0625) / 10)

      self.quantidadePedidos += 1

      if (self.estoqueLeiteDesnatadoEmbalado < _demand1*24.0 + _demand3*12.0 or self.estoqueLeiteSemidesnatadoEmbalado < _demand2*24.0 + _demand3*12.0 or self.estoqueAchocolatadoDesnatadoEmbalado < _demand4*120.0 + _demand6*60.0 or self.estoqueAchocolatadoSemidesnatadoEmbalado < _demand5*120.0 + _demand6*60.0):
        self.quantidadePedidosNãoAtendidos += 1
        if (self.estoqueLeiteDesnatadoEmbalado < _demand1*24.0 + _demand3*12.0 or self.estoqueLeiteSemidesnatadoEmbalado < _demand2*24.0 + _demand3*12.0):
          self.env.process(self.produzLeite(quantidadeLeiteDesnatado=np.max([0,self.ESTOQUE_MAXIMO_LEITE_DESNATADO_EMBALADO - self.estoqueLeiteDesnatadoEmbalado, 24*_demand1 + 12*_demand2 - self.estoqueLeiteDesnatadoEmbalado]), quantidadeLeiteSemidesnatado=np.max([0, self.ESTOQUE_MAXIMO_LEITE_SEMIDESNATADO_EMBALADO - self.estoqueLeiteSemidesnatadoEmbalado, _demand2*24 + 12*_demand3 - self.estoqueLeiteSemidesnatadoEmbalado])))

          self.custo += 24*(1 - 0.2)*1.5*(_demand1 + _demand2 + _demand3)
          self.custo += 24*(1.3 - 0.2)*1.5*(_demand4 + _demand5 + _demand6)

          continue
        if (self.estoqueAchocolatadoDesnatadoEmbalado < _demand4*120.0 + _demand6*60.0 or self.estoqueAchocolatadoSemidesnatadoEmbalado < _demand5*120.0 + _demand6*60.0):
          self.env.process(self.produzAchocolatado(quantidadeAchocolatadoDesnatado=np.max([0,self.ESTOQUE_MAXIMO_ACHOCOLATADO_DESNATADO_EMBALADO - self.estoqueAchocolatadoDesnatadoEmbalado]), quantidadeAchocolatadoSemidesnatado=np.max([0, self.ESTOQUE_MAXIMO_ACHOCOLATADO_SEMIDESNATADO_EMBALADO - self.estoqueAchocolatadoSemidesnatadoEmbalado])))

          self.custo += 24*(1 - 0.2)*1.5*(_demand1 + _demand2 + _demand3)
          self.custo += 24*(1.3 - 0.2)*1.5*(_demand4 + _demand5 + _demand6)

          continue


      else:
        self.cumpreComPedido(
            quantProduto1=np.floor(_demand1),
            quantProduto2=np.floor(_demand2),
            quantProduto3=np.floor(_demand3),
            quantProduto4=np.floor(_demand4),
            quantProduto5=np.floor(_demand5),
            quantProduto6=np.floor(_demand6)
        )



In [5]:
# _ = ((6200000 / (5*8)) / 24) / 4.0625

# print(np.floor(np.random.normal(_*1.5/4.0625, (_*1.5/4.0625) / 10)))
# print(np.floor(np.random.normal(_*1/4.0625, (_*1/4.0625) / 10)))
# print(np.floor(np.random.normal(_*.75/4.0625, (_*.75/4.0625) / 10)))
# print(np.floor(np.random.normal(_*.375/4.0625, (_*.375/4.0625) / 10)))
# print(np.floor(np.random.normal(_*.25/4.0625, (_*.25/4.0625) / 10)))
# print(np.floor(np.random.normal(_*.1875/4.0625, (_*.1875/4.0625) / 10)))

In [6]:
# def testADay():
#   def test(env2):
#     quantLeiteDesnatado = 0
#     quantLeiteSemidesnatado = 0
#     quantAchocolatadoDesnatado = 0
#     quantAchocolatadoSemidesnatado = 0
#     quantProd1 = 0
#     quantProd2 = 0
#     quantProd3 = 0
#     quantProd4 = 0
#     quantProd5 = 0
#     quantProd6 = 0
#     numVendas = 0
#     while True:
#       if (env2.now % (60*24) < 60*7):
#         yield env2.timeout(60*7 - env2.now % (60*24))
#         yield env2.timeout(np.random.exponential(scale=60))
#       elif (env2.now % (60*24) > 60*15):
#         yield env2.timeout(60*7 + 60*24 - env2.now % (60*24))
#         yield env2.timeout(np.random.exponential(scale=60))
#       yield env2.timeout(np.random.exponential(scale=60))
#       if (env2.now >= 60*15*5):
#         break
#       _ = ((6200000 / (5*8)) / 24) / 4.0625
#       prod1 = np.floor(np.random.normal((_*1.5/4.0625), (_*1.5/4.0625) / 10))
#       prod2 = np.floor(np.random.normal((_*1.0/4.0625), (_*1.0/4.0625) / 10))
#       prod3 = np.floor(np.random.normal((_*.75/4.0625), (_*.75/4.0625) / 10))
#       prod4 = np.floor(np.random.normal((_*.375/4.0625), (_*.375/4.0625) / 10))
#       prod5 = np.floor(np.random.normal((_*.25/4.0625), (_*.25/4.0625) / 10))
#       prod6 = np.floor(np.random.normal((_*.1875/4.0625), (_*.1875/4.0625) / 10))

#       quantProd1 += prod1
#       quantProd2 += prod2
#       quantProd3 += prod3
#       quantProd4 += prod4
#       quantProd5 += prod5
#       quantProd6 += prod6

#       quantLeiteDesnatado += quantProd1*24.0 + quantProd3*12.0
#       quantLeiteSemidesnatado += quantProd2*24.0 + quantProd3*12.0
#       quantAchocolatadoDesnatado += quantProd4*24.0 + quantProd6*12.0
#       quantAchocolatadoSemidesnatado += quantProd5*24.0 + quantProd6*12.0
#       numVendas += 1
#     # print(quantProd1)
#     # print(quantProd2)
#     # print(quantProd3)
#     # print(quantProd4)
#     # print(quantProd5)
#     # print(quantProd6)
#     # print()
#     # print(quantLeiteDesnatado)
#     # print(quantLeiteSemidesnatado)
#     # print(quantAchocolatadoDesnatado)
#     # print(quantAchocolatadoSemidesnatado)
#     print(quantLeiteDesnatado + quantLeiteSemidesnatado + quantAchocolatadoDesnatado + quantAchocolatadoSemidesnatado)
#     # print(numVendas)
#     # print()
#     # print('-----------------------')
#     # print()
#   env2 = simpy.Environment()
#   env2.process(test(env2))
#   env2.run(until=60*24*5)
# for _ in range(5000):
#   testADay()

In [7]:
def simulaUmaSemana(LIMITE_MÁXIMO_PASTEURIZADORA, FRAÇÃO_TAMANHO_ESTOQUE, PONTO_DE_ESTOQUE):

  env = simpy.Environment()
  solution = Solution(env,
                      estoqueInicialLeiteDesnatadoEmbalado=(3213531 + 2.58*456360) * FRAÇÃO_TAMANHO_ESTOQUE, #Em embalagens de 1L
                      # estoqueInicialLeiteDesnatadoEmbalado=np.math.inf, #Em embalagens de 1L
                      estoqueInicialLeiteSemidesnatadoEmbalado=(2353177 + 2.58*334466) * FRAÇÃO_TAMANHO_ESTOQUE, #Em embalagens de 1L
                      # estoqueInicialLeiteSemidesnatadoEmbalado=np.math.inf, #Em embalagens de 1L
                      estoqueInicialAchocolatadoDesnatadoEmbalado=5*(801352.3 + 2.58*114279) * FRAÇÃO_TAMANHO_ESTOQUE, #Em embalagens de 200ml
                      # estoqueInicialAchocolatadoDesnatadoEmbalado=np.math.inf, #Em embalagens de 200ml
                      estoqueInicialAchocolatadoSemidesnatadoEmbalado=5*(587995.3 + 2.58*83243.1) * FRAÇÃO_TAMANHO_ESTOQUE, #Em embalagens de 200ml
                      # estoqueInicialAchocolatadoSemidesnatadoEmbalado=np.math.inf, #Em embalagens de 200ml
                      estoqueInicialEmbalagemDeDesnatado=(3213531 + 2.58*456360) * FRAÇÃO_TAMANHO_ESTOQUE, #Em embalagens de 1l
                      # estoqueInicialEmbalagemDeDesnatado=np.math.inf, #Em embalagens de 1l
                      estoqueInicialEmbalagemDeSemidesnatado=(2353177 + 2.58*334466) * FRAÇÃO_TAMANHO_ESTOQUE, #Em embalagens de 1l
                      # estoqueInicialEmbalagemDeSemidesnatado=np.math.inf, #Em embalagens de 1l
                      estoqueInicialEmbalagensDeAchocolatadoDesnatado=5*(801352.3 + 2.58*114279) * FRAÇÃO_TAMANHO_ESTOQUE,
                      # estoqueInicialEmbalagensDeAchocolatadoDesnatado=np.math.inf,
                      estoqueInicialEmbalagensDeAchocolatadoSemidesnatado=5*(587995.3 + 2.58*83243.1) * FRAÇÃO_TAMANHO_ESTOQUE,
                      # estoqueInicialEmbalagensDeAchocolatadoSemidesnatado=np.math.inf,
                      estoqueInicialLeiteCru=(6956056 + 2.58*986030) * FRAÇÃO_TAMANHO_ESTOQUE, #Em litros
                      # estoqueInicialLeiteCru=np.math.inf, #Em litros
                      estoqueInicialAçucarEChocolate=(5*(801352.3 + 2.58*114279 + 587995.3 + 2.58*83243.1)) * FRAÇÃO_TAMANHO_ESTOQUE, #Em quantidades para fazer um achocolatado
                      # estoqueInicialAçucarEChocolate=np.math.inf, #Em quantidades para fazer um achocolatado
                      # LIMITE_MAQUINA_PASTEURIZADORA=700000, #Em litros de leite
                      LIMITE_MAQUINA_PASTEURIZADORA=LIMITE_MÁXIMO_PASTEURIZADORA, #Em litros de leite
                      getTempoPasteurização=lambda: np.random.triangular(28,30,35), #Processo de pasteurização lento
                      # getTempoLimparMaquinaPasteurização=lambda: np.random.triangular(2,3,5), #Em minutos
                      getTempoCentrifugaçãoSemidesnatado=lambda: np.random.triangular(8.5,9,10), #Um pouco mais que o te[orico para escoar o leite]
                      getTempoCentrifugaçãoDesnatado=lambda: np.random.triangular(15.5,16,17),
                      getTempoAdiçãoAçucarEChocolate=lambda: np.random.triangular(0.5,1,1),
                      getTempoHomogeneização=lambda: np.random.triangular(2,3,4),
                      getTempoEnvaseLeite=lambda quantidadeDeLeite: 80*quantidadeDeLeite/310000*np.random.normal(1,0.01),
                      getTempoEnvaseAchocolatado=lambda quantidadeDeAchocolatado: 5*quantidadeDeAchocolatado/630*np.random.normal(1,0.1),
                      getTempoDeEntregaLeiteCruComprado=lambda: np.max([30, np.random.normal(60*3,60)]),
                      quantidadeMaquinasPasteurizadoras=1,
                      quantidadeMaquinasCentrífugas=1,
                      quantidadeContainers=np.math.inf,
                      quantidadeMaquinasEnvasadoras=1,
                      getTempoDeEsperaEntreClientes=lambda: np.random.exponential(scale=20),
                      demandaSemanalLitrosDeLeite=6200000,
                      pontoDeRecompraDeEstoque=PONTO_DE_ESTOQUE
                      )
  env.run(until=5*24.0*60.0)
  solution.custo += solution.quantidadeDeCompras * 300

  if (solution.quantidadeDeCompras > 10):
    solution.custo += 500000
  # print(solution.quantidadeDeCompras)
  # print()
  # print()
  # print()
  # print(f'Foram vendidas {solution.quantidadeVendidoProduto1} unidades do produto 1')
  # print(f'Foram vendidas {solution.quantidadeVendidoProduto2} unidades do produto 2')
  # print(f'Foram vendidas {solution.quantidadeVendidoProduto3} unidades do produto 3')
  # print(f'Foram vendidas {solution.quantidadeVendidoProduto4} unidades do produto 4')
  # print(f'Foram vendidas {solution.quantidadeVendidoProduto5} unidades do produto 5')
  # print(f'Foram vendidas {solution.quantidadeVendidoProduto6} unidades do produto 6')
  # print()
  # print(solution.quantidadeVendidoProduto5*24 + solution.quantidadeVendidoProduto6*12)
  # print(f'Foram vendidos {solution.quantidadeVendidoProduto1*24 + solution.quantidadeVendidoProduto3*12} litros de leite desnatado')
  # print(f'Foram vendidos {solution.quantidadeVendidoProduto2*24 + solution.quantidadeVendidoProduto3*12} litros de leite semidesnatado')
  # print(f'Foram vendidos {solution.quantidadeVendidoProduto4*24 + solution.quantidadeVendidoProduto6*12} litros de achocolatado desnatado')
  # print(f'Foram vendidos {solution.quantidadeVendidoProduto5*24 + solution.quantidadeVendidoProduto6*12} litros de achocolatado semidesnatado')
  # print()
  # print(24*(solution.quantidadeVendidoProduto1 + solution.quantidadeVendidoProduto2 + solution.quantidadeVendidoProduto3 + solution.quantidadeVendidoProduto4 + solution.quantidadeVendidoProduto5 + solution.quantidadeVendidoProduto6))
  # print(f'{solution.quantidadePedidosNãoAtendidos} --- {solution.quantidadePedidos} --- {100*solution.quantidadePedidosNãoAtendidos/solution.quantidadePedidos}% ---- {24*(solution.quantidadeVendidoProduto1 + solution.quantidadeVendidoProduto2 + solution.quantidadeVendidoProduto3 + solution.quantidadeVendidoProduto4 + solution.quantidadeVendidoProduto5 + solution.quantidadeVendidoProduto6)}')
  # print(solution.quantidadePedidos)
  # print((solution.quantidadePedidos - solution.quantidadePedidosNãoAtendidos) / solution.quantidadePedidos)
  # print(f'A quantidade de atraso em minutos foi: {solution.quantidadeAtrasoEmMinutos} em {solution.quantidadePedidosAtrasados} pedidos')
  # print(solution.quantidadePedidosAtrasados)
  # print(solution.quantidadeAtrasoEmMinutos)
  # print(f'A quantidade de compras foi de: {solution.quantidadeDeCompras}')
  # print(solution.quantidadeDeCompras)
  # print(f'Os produtos foram produzidos em: {solution.quantidadeDeBateladas} bateladas')
  # print(solution.quantidadeDeBateladas)
  # print(solution.quantidadeDeVendas)
  # print(solution.tempoAtrasoDaVenda)
  # print()
  # print('------------------------------------------------------------------------------------------------------------')
  # print()
  return {
      'vendasProduto1': solution.quantidadeVendidoProduto1,
      'vendasProduto2': solution.quantidadeVendidoProduto2,
      'vendasProduto3': solution.quantidadeVendidoProduto3,
      'vendasProduto4': solution.quantidadeVendidoProduto4,
      'vendasProduto5': solution.quantidadeVendidoProduto5,
      'vendasProduto6': solution.quantidadeVendidoProduto6,
      'nivelAtendimento': (solution.quantidadePedidos - solution.quantidadePedidosNãoAtendidos) / solution.quantidadePedidos,
      'custo': solution.custo
  }

limitesPasteurizadora = [25000, 40000, 70000, 100000, 150000]
fraçõesEstoque = [.35, .5, 2/3, .775, .9]
pontosEstoque = [.5, .55, 2/3, .775, .9]

with open ('/content/response.csv', 'w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['limite_pasteurizadora', 'tamanho_dos_estoques', 'ponto_do_estoque', 'custo', 'nivel_de_atendimento'])
  medias_importantes = {
      '000': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '001': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '002': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '003': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '004': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '010': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '011': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '012': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '013': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '014': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '020': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '021': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '022': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '023': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '024': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '030': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '031': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '032': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '033': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '034': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '040': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '041': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '042': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '043': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '044': {
        'custo': 0,
        'nivelAtendimento': 0
      },


      '100': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '101': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '102': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '103': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '104': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '110': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '111': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '112': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '113': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '114': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '120': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '121': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '122': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '123': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '124': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '130': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '131': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '132': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '133': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '134': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '140': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '141': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '142': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '143': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '144': {
        'custo': 0,
        'nivelAtendimento': 0
      },


      '100': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '101': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '102': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '103': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '104': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '110': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '111': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '112': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '113': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '114': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '120': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '121': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '122': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '123': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '124': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '130': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '131': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '132': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '133': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '134': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '140': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '141': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '142': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '143': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '144': {
        'custo': 0,
        'nivelAtendimento': 0
      },


      '200': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '201': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '202': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '203': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '204': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '210': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '211': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '212': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '213': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '214': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '220': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '221': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '222': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '223': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '224': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '230': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '231': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '232': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '233': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '234': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '240': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '241': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '242': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '243': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '244': {
        'custo': 0,
        'nivelAtendimento': 0
      },


      '300': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '301': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '302': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '303': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '304': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '310': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '311': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '312': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '313': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '314': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '320': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '321': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '322': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '323': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '324': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '330': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '331': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '332': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '333': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '334': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '340': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '341': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '342': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '343': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '344': {
        'custo': 0,
        'nivelAtendimento': 0
      },


      '400': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '401': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '402': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '403': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '404': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '410': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '411': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '412': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '413': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '414': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '420': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '421': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '422': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '423': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '424': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '430': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '431': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '432': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '433': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '434': {
        'custo': 0,
        'nivelAtendimento': 0
      },

      '440': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '441': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '442': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '443': {
        'custo': 0,
        'nivelAtendimento': 0
      },
      '444': {
        'custo': 0,
        'nivelAtendimento': 0
      }
  }
  for limiteIndex in range(len(limitesPasteurizadora)):
    for fraçãoIndex in range(len(fraçõesEstoque)):
      for pontoIndex in range(len(pontosEstoque)):
        for _ in range(300):
          simulation = simulaUmaSemana(LIMITE_MÁXIMO_PASTEURIZADORA=limitesPasteurizadora[limiteIndex],
                          FRAÇÃO_TAMANHO_ESTOQUE=fraçõesEstoque[fraçãoIndex],
                          PONTO_DE_ESTOQUE=pontosEstoque[pontoIndex])
          medias_importantes[f'{limiteIndex}{fraçãoIndex}{pontoIndex}']['custo'] += simulation['custo'] / 300
          medias_importantes[f'{limiteIndex}{fraçãoIndex}{pontoIndex}']['nivelAtendimento'] += simulation['nivelAtendimento'] / 300
          writer.writerow([f'level: {limiteIndex}',
                           f'level: {fraçãoIndex}',
                           f'level: {pontoIndex}',
                           simulation['custo'],
                           simulation['nivelAtendimento']])

    print('Foi un poko')


# for _ in range(300):
#   simulaUmaSemana()


Foi un poko
Foi un poko
Foi un poko
Foi un poko
Foi un poko


In [8]:
print(sorted(medias_importantes.keys(), key=lambda x: medias_importantes[x]['custo']))

['141', '241', '240', '243', '140', '340', '042', '441', '442', '341', '242', '342', '043', '142', '143', '440', '343', '041', '443', '040', '433', '232', '431', '331', '432', '031', '132', '430', '131', '030', '230', '032', '130', '033', '233', '231', '333', '133', '332', '330', '244', '144', '044', '134', '444', '034', '423', '344', '120', '434', '022', '234', '121', '334', '322', '321', '222', '221', '420', '223', '421', '422', '220', '023', '323', '320', '122', '124', '021', '123', '020', '424', '224', '324', '024', '011', '110', '012', '211', '010', '210', '312', '112', '411', '412', '414', '310', '311', '413', '111', '213', '212', '410', '313', '014', '013', '214', '114', '113', '314', '404', '403', '000', '200', '401', '302', '303', '204', '402', '304', '101', '203', '004', '202', '001', '104', '300', '102', '003', '400', '103', '002', '301', '100', '201']


In [21]:
print(medias_importantes['141'])

{'custo': 2464325.925822438, 'nivelAtendimento': 0.9986042234442363}


In [22]:
print(medias_importantes['241'])

{'custo': 2474829.61650857, 'nivelAtendimento': 0.9984424842517026}


In [23]:
print(medias_importantes['240'])

{'custo': 2479379.035208048, 'nivelAtendimento': 0.9980586482632517}


In [24]:
print(medias_importantes['243'])

{'custo': 2484234.956892831, 'nivelAtendimento': 0.9991744971901789}


In [28]:
print(medias_importantes['223'])

{'custo': 3102777.1151705547, 'nivelAtendimento': 0.9331834238546568}


In [29]:
print(medias_importantes['123'])

{'custo': 3226843.5568467886, 'nivelAtendimento': 0.9161870369111357}


In [36]:
print(medias_importantes['023'])

{'custo': 3129004.1569611747, 'nivelAtendimento': 0.9243649168461371}
